# Exploring the billing datasets

```Do not run in poetry shell```

In [1]:
# Imports
import os, sys
sys.path.insert(0,'/mnt/c/dev/cl/pipeline')
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import BlobClient
from io import StringIO
from src.config import My_Config as cfg
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport
import seaborn as sns
import pandas as pd
import numpy as np

/home/tj/.cache/pypoetry/virtualenvs/pipeline-MexhjMpT-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set variables
CONNECTION_STRING = cfg.storage_connection_string()
CONTAINERNAME = cfg.storage_container_name_1()
BLOBNAME = cfg.storage_blob_name()

In [3]:
# List files from blob storage
def list_blobs_in_container():
    from azure.storage.blob import BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(CONTAINERNAME)
    blobs_list = container_client.list_blobs()
    for blob in blobs_list:
        print(blob.name + '\n')
    return blobs_list

list_blobs_in_container()

AWS-Billing-Data.parquet

Azure-Billing-Data.parquet

GCP-Billing-Data.parquet



<iterator object azure.core.paging.ItemPaged at 0x7f631dfdd490>

In [4]:
def create_sas():
    CONTAINERNAME = cfg.storage_container_name_1()
    BLOBNAME = cfg.storage_blob_name()
    ACCOUNTNAME = cfg.storage_account_name()
    QUERYSTRING = cfg.blob_query_string()
    
    sas_url = f"https://{ACCOUNTNAME}.blob.core.windows.net/{CONTAINERNAME}/{BLOBNAME}{QUERYSTRING}"
    return sas_url
 
    
sas_url = create_sas()

In [5]:
df = pd.read_parquet(sas_url)
df_len, df_width = df.shape
print('Rows: ' +str(df_len),'\nColumns: ' + str(df_width))

Rows: 626 
Columns: 18


In [6]:
for column in df:
    x = df
    x = x[column].drop_duplicates().dropna()
    print("The column '" + column + "' has " + str(x.shape[0]) + " unique rows")

The column 'billing_account_id' has 1 unique rows
The column 'service' has 3 unique rows
The column 'sku' has 7 unique rows
The column 'usage_start_time' has 308 unique rows
The column 'usage_end_time' has 308 unique rows
The column 'project' has 626 unique rows
The column 'labels' has 331 unique rows
The column 'system_labels' has 626 unique rows
The column 'location' has 3 unique rows
The column 'export_time' has 110 unique rows
The column 'cost' has 1 unique rows
The column 'currency' has 1 unique rows
The column 'currency_conversion_rate' has 1 unique rows
The column 'usage' has 298 unique rows
The column 'credits' has 626 unique rows
The column 'invoice' has 1 unique rows
The column 'cost_type' has 1 unique rows
The column 'adjustment_info' has 0 unique rows


In [7]:
df.describe()

,cost,currency_conversion_rate
count,626.0,626.0000
mean,0.0,9.8245
std,0.0,0.0000
min,0.0,9.8245
25%,0.0,9.8245
50%,0.0,9.8245
75%,0.0,9.8245
max,0.0,9.8245


In [8]:
LOCAL_FILE_PATH = cfg.local_files_path()
profile = ProfileReport(df, title="GCP Billing Report")
profile.to_file(LOCAL_FILE_PATH + "reports/GCP-Billing-Report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/dev/cl/pipeline/local/reports/GCP-Billing-Report.html'